In [ ]:
from pathlib import Path

root_dir = Path("data", "adaptation")
static_path = root_dir.joinpath("static")
hazard_path =static_path.joinpath("hazard")
network_path = static_path.joinpath("network")
output_path=root_dir.joinpath("output")
tif_files = list(hazard_path.glob("*.tif"))

In [ ]:
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,HazardSection,)
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkSection)
from ra2ce.analysis.damages.damages import AnalysisSectionDamages
from ra2ce.analysis.analysis_config_data.analysis_config_data import (AnalysisConfigData,ProjectSection,AnalysisSectionBase,AnalysisSectionLosses)

In [ ]:

_network_section = NetworkSection(
    source= SourceEnum.SHAPEFILE,       
    primary_file = [network_path.joinpath("network.shp")], 
    file_id = "ID",
    link_type_column='highway',
    save_gpkg=True
)

_hazard = HazardSection(
    hazard_map=[Path(file) for file in hazard_path.glob("*.tif")],
    hazard_field_name= ['waterdepth'],
    aggregate_wl = AggregateWlEnum.MAX,
    hazard_crs = 'EPSG:4326' 
)


_network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=static_path,
    output_path=output_path,
    network=_network_section,
    hazard=_hazard
    )

In [ ]:

handler = Ra2ceHandler.from_config(_network_config_data, None)

handler.configure()
handler.run_analysis()

In [ ]:
from ra2ce.analysis.analysis_config_data.analysis_config_data import (AnalysisSectionAdaptation,AnalysisSectionAdaptationOption)
from ra2ce.analysis.analysis_config_data.enums.analysis_enum import AnalysisEnum
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import AnalysisLossesEnum

adaptation_op0 = [AnalysisSectionAdaptationOption(
    id="AO0",
    name="no adaptation",
)]

adaptation_op1 = [AnalysisSectionAdaptationOption(
    id="AO1",
    name="Adaptation_option_1",
    construction_cost=1000,
    maintenance_interval=3,
    maintenance_cost=200
)]

adaptation_op2 = [AnalysisSectionAdaptationOption(
    id="AO2",
    name="Adaptation_option_2",
    construction_cost=5000,
    maintenance_interval=3,
    maintenance_cost=50
)]

adaptation = [AnalysisSectionAdaptation(
    analysis=AnalysisEnum.ADAPTATION,
    losses_analysis=AnalysisLossesEnum.SINGLE_LINK_LOSSES,
    discount_rate=0.0,
    time_horizon=0.0,
    #CBA_time_horizon=0.0,
    #vat=0.0, - not included
    climate_factor=0.0,
    initial_frequency=0.0,
    no_adaptation_option=AnalysisSectionAdaptationOption(adaptation_op0),
    adaptation_options=[adaptation_op1, adaptation_op2]
)]

_analysis_config_data = AnalysisConfigData(
    analyses= adaptation,
    root_path= root_dir,
    output_path= output_path,
    )

In [ ]:
handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)

handler.configure()
handler.run_analysis()